In [21]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F



In [22]:
# Check if CUDA is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [23]:
# Define a simple CNN model from scratch
class WasteClassifierCNN(nn.Module):
    def __init__(self, num_classes):
        super(WasteClassifierCNN, self).__init__()
        # Define layers
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # Input: 3x224x224 → Output: 16x224x224
        self.pool = nn.MaxPool2d(2, 2)               # Output: 16x112x112
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) # Output: 32x112x112 → pool → 32x56x56
        self.fc1 = nn.Linear(32 * 56 * 56, 128)      # Flatten and FC layer
        self.fc2 = nn.Linear(128, num_classes)       # Output layer

    def forward(self, x):
        # Forward pass
        x = self.pool(F.relu(self.conv1(x)))  # Conv1 + ReLU + Pool
        x = self.pool(F.relu(self.conv2(x)))  # Conv2 + ReLU + Pool
        x = x.view(-1, 32 * 56 * 56)          # Flatten to 1D vector
        x = F.relu(self.fc1(x))               # FC1 + ReLU
        x = self.fc2(x)                       # Output layer
        return x


In [24]:
# Define data transformations for training and testing
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Resize to 224x224 for the input images
    transforms.ToTensor(),           # Convert images to tensor
])

# Load the dataset using ImageFolder
train_dataset = datasets.ImageFolder('NDATA/DATASET/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Optionally, load the test dataset for evaluation
test_dataset = datasets.ImageFolder('NDATA/DATASET/train', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [25]:
# Initialize the model
num_classes = len(train_dataset.classes)  # Get number of classes from the dataset
model = WasteClassifierCNN(num_classes).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load dataset
train_dataset = datasets.ImageFolder('NDATA/DATASET/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define CNN Model
class WasteClassifierCNN(nn.Module):
    def __init__(self, num_classes):
        super(WasteClassifierCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 3x224x224 → 16x112x112
        x = self.pool(F.relu(self.conv2(x)))  # 16x112x112 → 32x56x56
        x = x.view(-1, 32 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate model
num_classes = len(train_dataset.classes)
model = WasteClassifierCNN(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [27]:
# Training loop
for epoch in range(7):  # You can increase the number of epochs as needed
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    # Loop through batches of data
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass

        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize the weights

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Accumulate the loss for averaging
        running_loss += loss.item()

    # Calculate average loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
 
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")


Epoch 1, Loss: 0.6302, Accuracy: 73.70%
Epoch 2, Loss: 0.5323, Accuracy: 78.18%
Epoch 3, Loss: 0.4501, Accuracy: 81.70%
Epoch 4, Loss: 0.3521, Accuracy: 85.84%
Epoch 5, Loss: 0.2445, Accuracy: 90.40%
Epoch 6, Loss: 0.1577, Accuracy: 94.07%
Epoch 7, Loss: 0.1034, Accuracy: 96.29%


In [28]:
# Evaluation after training
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # No need to compute gradients during evaluation
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)  # Forward pass
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate final accuracy on the test set
test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 98.25%


In [29]:
# Save the trained model
torch.save(model.state_dict(), 'waste_combined.pth')
